In [ ]:
!pip install --user -r requirements.txt
import sys
print(sys.executable)


In [1]:
from transformers import AutoTokenizer, BlipForQuestionAnswering, BlipConfig
import torch
print('working')
# Load the BLIP model and tokenizer
model_name = "Salesforce/blip-vqa-base"
tokenizer = AutoTokenizer.from_pretrained(model_name) 
config = BlipConfig.from_pretrained(model_name)
model = BlipForQuestionAnswering.from_pretrained(model_name, config=config)


C:\Users\waltw\MiniConda3\envs\myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


working


In [2]:
import torch

print(torch.version.cuda)


11.3


In [3]:
import json

# Load the answer JSON file
answer_file = r"C:\Users\waltw\New_VQA\v2_Annotations_Train_mscoco\v2_mscoco_train2014_annotations.json"
with open(answer_file) as f:
    answer_data = json.load(f)

# Create a mapping of question IDs to questions
question_text_file = r"C:\Users\waltw\New_VQA\2017 overlied\train_question\train_Q.txt"
question_mapping = {}
with open(question_text_file) as f:
    for line in f:
        question_data = json.loads(line)
        question_mapping[question_data['question_id']] = question_data['question']

# Now, let's preprocess the data and match the questions to the answers
preprocessed_data = []
for annotation in answer_data['annotations']:
    image_id = annotation['image_id']
    question_id = annotation['question_id']
    question = question_mapping.get(question_id, 'Unknown Question')
    answer = annotation['multiple_choice_answer']
    preprocessed_data.append({'image_id': image_id, 'question': question, 'answer': answer, 'question_id': question_id})


# Now preprocessed_data contains the matched questions, question IDs, and answers


In [ ]:
print(preprocessed_data)

In [4]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("GPU is available and will be used.")
else:
    device = torch.device('cpu')
    print("GPU is not available. The computation will be done on CPU.")

# The following line will show the name of the GPU
!nvidia-smi


GPU is available and will be used.
Wed Aug 16 19:33:45 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.14                 Driver Version: 531.14       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 T...  WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   63C    P8               13W /  N/A|      0MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+--------------------------------

In [5]:
try:
    from transformers import AutoTokenizer, BlipForQuestionAnswering, BlipProcessor, BlipConfig
    from PIL import Image
    import os
except ImportError:
    print("Some required packages are not found. Installing them...")
    try:
        import subprocess
        subprocess.check_call(['pip', 'install', 'transformers', 'PILLOW'])
        print("Packages installed successfully.")
    except Exception as e:
        print("Error installing packages:", e)
        # You can exit or handle the error as needed

# Now you can continue with your code


In [6]:
import transformers
transformers.__version__


'4.31.0'

In [7]:
import torch
from transformers import AutoTokenizer, BlipForQuestionAnswering, BlipProcessor, BlipConfig
from PIL import Image
import os
import shutil

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    raise EnvironmentError("CUDA is not available. Please make sure you have a compatible GPU and the necessary drivers installed.")

# Load the BLIP model and tokenizer
model_name = "Salesforce/blip-vqa-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = BlipConfig.from_pretrained(model_name)
model = BlipForQuestionAnswering.from_pretrained(model_name, config=config).to(device)

# Initialize the image processor
processor = BlipProcessor.from_pretrained(model_name)

# Define your training loop here and use the preprocessed_data to train the model
# Remember to tokenize the questions and answers using the tokenizer and convert them to tensors

base_image_path = r"C:\Users\waltw\New_VQA\2017 overlied\train"

# Define the optimizer and other hyperparameters for training
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
num_epochs = 5

# Specify the directory to save checkpoints
model_checkpoint = r"C:\Users\waltw\New_VQA\Clip_Vil"

# Initialize the count for training successes and checkpoints
training_success_count = 0
checkpoint_interval = 100
max_checkpoints_to_keep = 3  # You can change this to control the number of checkpoints to keep

# Example fine-tuning loop
for epoch in range(num_epochs):
    for data in preprocessed_data:
        question = data['question']
        answer = data['answer']
        image_id = data['image_id']
        question_id = data['question_id']

        # Convert image_id to a 12-digit string with leading zeros
        image_id_str = str(image_id).zfill(12)

        # Generate the image file name
        image_file_name = f'COCO_train2014_{image_id_str}.jpg'
        image_path = os.path.join(base_image_path, image_file_name)

        try:
            # Load and preprocess the image
            image = Image.open(image_path).convert('RGB')
            inputs = processor(images=image, questions=question, return_tensors="pt")

            # Extract the image features
            pixel_values = inputs.pixel_values.to(device)

            # Tokenize the question and answer
            encoded = tokenizer(question, answer, return_tensors="pt")
            input_ids = encoded['input_ids'].to(device)
            attention_mask = encoded['attention_mask'].to(device)

            # Convert the answer to a tensor
            label = torch.tensor([tokenizer.encode(answer, add_special_tokens=False)], dtype=torch.long).to(device)

            # Fine-tuning step
            outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask, labels=label)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # Increment the training success count
            training_success_count += 1

            # Check if it's time to save a checkpoint
            if training_success_count % checkpoint_interval == 0:
                # Save the fine-tuned model checkpoint
                checkpoint_dir = os.path.join(model_checkpoint, f'checkpoint_{training_success_count}')
                model.save_pretrained(checkpoint_dir)
                print(f"Checkpoint saved at training success: {training_success_count}")

                # Print out the last training success image
                print(f"Last training success image: {image_path}")

                # Delete older checkpoints if there are more than max_checkpoints_to_keep
                if training_success_count > max_checkpoints_to_keep * checkpoint_interval:
                    old_checkpoint = os.path.join(model_checkpoint, f'checkpoint_{training_success_count - max_checkpoints_to_keep * checkpoint_interval}')
                    shutil.rmtree(old_checkpoint)
                    print(f"Old checkpoint removed: {old_checkpoint}")

        except FileNotFoundError:
            # Skip the file if it's not found
            pass


Checkpoint saved at training success: 100
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000262261.jpg
Checkpoint saved at training success: 200
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000524651.jpg
Checkpoint saved at training success: 300
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000510665.jpg
Checkpoint saved at training success: 400
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000524389.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_100
Checkpoint saved at training success: 500
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000327369.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_200
Checkpoint saved at training success: 600
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014

Checkpoint saved at training success: 4000
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000139240.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_3700
Checkpoint saved at training success: 4100
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000139429.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_3800
Checkpoint saved at training success: 4200
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000008553.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_3900
Checkpoint saved at training success: 4300
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000533049.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_4000
Checkpoint saved at training success: 4400
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_trai

Checkpoint saved at training success: 7800
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000277998.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_7500
Checkpoint saved at training success: 7900
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000147192.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_7600
Checkpoint saved at training success: 8000
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000278467.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_7700
Checkpoint saved at training success: 8100
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000409740.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_7800
Checkpoint saved at training success: 8200
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_trai

Checkpoint saved at training success: 11600
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000003992.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_11300
Checkpoint saved at training success: 11700
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000286359.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_11400
Checkpoint saved at training success: 11800
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000024436.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_11500
Checkpoint saved at training success: 11900
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000288099.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_11600
Checkpoint saved at training success: 12000
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\

Checkpoint saved at training success: 15400
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000032893.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_15100
Checkpoint saved at training success: 15500
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000293524.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_15200
Checkpoint saved at training success: 15600
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000295461.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_15300
Checkpoint saved at training success: 15700
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000033494.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_15400
Checkpoint saved at training success: 15800
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\

...

Checkpoint saved at training success: 272900
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000088931.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_272600
Checkpoint saved at training success: 273000
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000089115.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_272700
Checkpoint saved at training success: 273100
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000068442.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_272800
Checkpoint saved at training success: 273200
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000351686.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_272900
Checkpoint saved at training success: 273300
Last training success image: C:\Users\waltw\New_VQA\2017 overli

Checkpoint saved at training success: 276600
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000038305.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_276300
Checkpoint saved at training success: 276700
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000431553.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_276400
Checkpoint saved at training success: 276800
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000492268.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_276500
Checkpoint saved at training success: 276900
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000361356.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_276600
Checkpoint saved at training success: 277000
Last training success image: C:\Users\waltw\New_VQA\2017 overli

Checkpoint saved at training success: 280300
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000106964.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_280000
Checkpoint saved at training success: 280400
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000369357.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_280100
Checkpoint saved at training success: 280500
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000500646.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_280200
Checkpoint saved at training success: 280600
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000107691.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_280300
Checkpoint saved at training success: 280700
Last training success image: C:\Users\waltw\New_VQA\2017 overli

Checkpoint saved at training success: 284000
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000377545.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_283700
Checkpoint saved at training success: 284100
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000377778.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_283800
Checkpoint saved at training success: 284200
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000115864.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_283900
Checkpoint saved at training success: 284300
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000247157.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_284000
Checkpoint saved at training success: 284400
Last training success image: C:\Users\waltw\New_VQA\2017 overli

Checkpoint saved at training success: 287700
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000124751.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_287400
Checkpoint saved at training success: 287800
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000275795.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_287500
Checkpoint saved at training success: 287900
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000125188.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_287600
Checkpoint saved at training success: 288000
Last training success image: C:\Users\waltw\New_VQA\2017 overlied\train\COCO_train2014_000000125351.jpg
Old checkpoint removed: C:\Users\waltw\New_VQA\Clip_Vil\checkpoint_287700
Checkpoint saved at training success: 288100
Last training success image: C:\Users\waltw\New_VQA\2017 overli

### Failed

In [ ]:
import os
import requests
import torch
from PIL import Image
from transformers import BlipForQuestionAnswering, BlipProcessor, BlipConfig

# Define the path to the folder containing the images
image_folder = '/content/drive/MyDrive/Colab Notebooks/VQA+Gaze/2017 overlied/train'

# Read the train_Q.txt file and extract image IDs and questions
train_q_file = '/content/drive/MyDrive/Colab Notebooks/VQA+Gaze/2017 overlied/train_question/train_Q.txt'
with open(train_q_file, 'r') as f:
    questions_data = [eval(line) for line in f]

# Define the question to ask about the images
question_prefix = "How many objects are in the picture?"

# Load the BLIP model architecture
config = BlipConfig.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering(config)

# Load the fine-tuned model checkpoint using torch.load
model_checkpoint = '/content/drive/MyDrive/Colab Notebooks/VQA+Gaze/model_base_capfilt_large.pth'
model.load_state_dict(torch.load(model_checkpoint, map_location=torch.device('cpu')))

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# Loop through all the image files and test the model
for img_file in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_file)

    # Extract the last 12 digits from the image file name
    image_id = int(img_file[-12:])

    # Find the corresponding question for the image ID in questions_data
    question_data = next((q for q in questions_data if q["image_id"] == image_id), None)
    if question_data is not None:
        question = question_data["question"]

        # Preprocess the image and question for the model
        raw_image = Image.open(img_path).convert('RGB')
        inputs = processor(raw_image, question_prefix + " " + question, return_tensors="pt")

        # Generate the answer using the fine-tuned model
        out = model.generate(**inputs)

        # Decode and print the answer
        answer = processor.decode(out[0], skip_special_tokens=True)
        print(f"Image: {img_file}, Question: {question}, Answer: {answer}")
